In [2]:
import subprocess 

username = 'PikaMoonCoin'
subprocess.run(["python", "selenium-twitter-scraper/scraper", "--tweets=10", f"--username={username}"])

CompletedProcess(args=['python', 'selenium-twitter-scraper/scraper', '--tweets=10', '--username=PikaMoonCoin'], returncode=1)

In [4]:
import pandas as pd
from datetime import datetime
import ast

now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d_%H-%M-%S")
formatted_now = '2024-07-16_12-19-11'
df = pd.read_csv(f"tweets/{formatted_now}_tweets_1-10.csv")
tweets, hashtags, dates = [], [], []
for i, row in df.iterrows():
    tweets.append(row[4])
    hashtags.append(ast.literal_eval(row[9]))
    dates.append(row[2][:10])
dates


C:\Users\anush\AppData\Local\Temp\ipykernel_13440\1455726177.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tweets.append(row[4])
C:\Users\anush\AppData\Local\Temp\ipykernel_13440\1455726177.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hashtags.append(ast.literal_eval(row[9]))
C:\Users\anush\AppData\Local\Temp\ipykernel_13440\1455726177.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dates.append(row[2][:10])


['2024-04-19',
 '2024-07-15',
 '2024-07-15',
 '2024-07-15',
 '2024-07-14',
 '2024-07-12',
 '2024-07-11',
 '2024-07-10',
 '2024-07-09',
 '2024-07-08']

In [5]:
from collections import Counter

total_lists = len(hashtags)
half_lists = total_lists // 4
hashtag_counts = Counter()
for hashtags_l in hashtags:
    unique_hashtags = set(hashtags_l)
    hashtag_counts.update(unique_hashtags)
frequent_hashtags = [hashtag for hashtag, count in hashtag_counts.items() if count >= half_lists]
frequent_hashtags


['#Pikamoon']

In [6]:
for tag in frequent_hashtags:
    subprocess.run(["python", "selenium-twitter-scraper/scraper", "--tweets=10", f"--hashtag={tag[1:]}"])
    now = datetime.now()
    formatted_now = now.strftime("%Y-%m-%d_%H-%M-%S")
    formatted_now = '2024-07-16_12-19-11'
    df = pd.read_csv(f"tweets/{formatted_now}_tweets_1-10.csv")
    for i, row in df.iterrows():
        tweets.append(row[4])


NameError: name 'subprocess' is not defined

In [7]:
raw_tweets = 'Here are the last few posts from a crypto token & its followers:\n'
for i in tweets:
    print(i)
    raw_posts = raw_posts + i +"\n-\n"
    print('---------')

#Pikamoon: Openworld Trailer (Teaser) $PIKA is a movement.

The #PIKAARMY march together everyday.

Our game will be AAA 
---------
Team Meal at Orbit Cosmos Tonight  

Let’s eat up and keep fuelling  
---------
Good morning, #Pikamoon community!

We're all at our development office in Pakistan!Monday Giveaway ($250 of $PIKA) 

To participate:Follow   
---------
We're also filming a behind-the-scenes documentary of this trip to further demonstrate our transparency and dedication to the future.

Stay tuned, PIKA Army!
---------
Here's another sneak peek of the #Pikamoon Open World!

We're making tremendous progress every week 
---------
Keep bringing us that $PIKA stuff 
---------
GM from  

Meet some of the team:Muhammad Umair (3D Modeler) Hamza Siddiqui (CG Generalist) Salik Saeed (Animator)Jazib Tasaduq (Managing Director)$100 GIVEAWAY
---------
We're excited to share a very early version of the #PIKAMOON open world with you!

Please note, this post is solely to showcase our developm

In [4]:
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer
model_name = "AfterRain007/cryptobertRefined"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=128, truncation=True, padding = 'max_length')
preds = pipe(tweets)
preds

c:\Users\anush\OneDrive\Desktop\Work\smallcap\scraper-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'label': 'Neutral', 'score': 0.6220372915267944},
 {'label': 'Bullish', 'score': 0.9156750440597534},
 {'label': 'Neutral', 'score': 0.5735231041908264},
 {'label': 'Bullish', 'score': 0.5215172171592712},
 {'label': 'Neutral', 'score': 0.7277060151100159},
 {'label': 'Bullish', 'score': 0.9500432014465332},
 {'label': 'Neutral', 'score': 0.7635330557823181},
 {'label': 'Neutral', 'score': 0.5506970286369324},
 {'label': 'Bullish', 'score': 0.936409592628479},
 {'label': 'Neutral', 'score': 0.6510459780693054}]

In [6]:
filtered_tweets = []
for i in range (len(tweets)):
    if preds[i]['score'] > 0.7 and preds[i]['label'] in ['Bearish', 'Bullish']:
        filtered_tweets.append(tweets[i])
filtered_tweets


['Team Meal at Orbit Cosmos Tonight  \n\nLet’s eat up and keep fuelling  ',
 'Keep bringing us that $PIKA stuff ',
 'Pikamoon began as a mere idea back in October 2022…\n\nThree friends, Brock, Kanto, and Nix, sat envisioning how to revolutionise the P2E gaming world. \n\nThe bull market was behind us, nothing (literally nothing!) was happening, yet Pikamoon has since blossomed into what we see']

In [23]:
from datetime import datetime
first = datetime.strptime(dates[0], "%Y-%m-%d")
last = datetime.strptime(dates[9], "%Y-%m-%d")
diff_d = abs((first-last).days)
rate_d = len(dates) / diff_d if diff_d > 0 else len(dates)

post_rate = f'''The 10 most recent tweets from the given user ranges between {dates[0]} to {dates[9]}, which gives
an average posts per month rate of {rate_d*30}. '''.replace('\n', ' ')
post_rate

'The 10 most recent tweets from the given user ranges between 2024-04-19 to 2024-07-08, which gives an average posts per month rate of 3.75'

In [28]:
# Count occurrences of 'bearish' and 'bullish'
bearish_count = sum(1 for item in preds if item['label'] == 'Bearish')
bullish_count = sum(1 for item in preds if item['label'] == 'Bullish')
sent_perc = f'''Out of the posts collected, our sentiment analysis model found {int(bullish_count/len(tweets)*100)}% 
of the posts to be positive, and {int(bearish_count/len(tweets)*100)}% negative. '''.replace('\n', ' ')
sent_perc


'Out of the posts collected, our sentiment analysis model found 40%  of the posts to be positive, and 0% negative. '

In [1]:
import json, os, requests
from urllib.request import urlopen
from dotenv import load_dotenv
load_dotenv()

cname = 'pikamoonofficial'
url = f"https://api.telegram.org/bot{os.environ.get('token1')}:{os.environ.get('token2')}/getChatMemberCount?chat_id=@pikamoonofficial"
with urlopen(url) as f:
    resp = json.load(f)

print(resp['result'])

17788


In [2]:
import json
# Load the JSON data from the file
with open('channel_messages.json', 'r') as file:
    data = json.load(file)
# Loop over the JSON object content
raw_posts = 'Here are the last few posts from a crypto token:\n'
for entry in data[:5]:
    # Print the desired values
    # print(f"Date: {entry['date']}")
    # print(f"Message: {entry['message']}")
    # print(f"Desc: {entry['description']}")
    # print("---------------------------")
    raw_posts = raw_posts + entry['message'].strip()+"\n-\n"
raw_posts

---------------------------
---------------------------
---------------------------
---------------------------
---------------------------


'Here are the last few posts from a crypto token:\nAnother Sneak Peak 👀\n\nhttps://x.com/pikamooncoin/status/1814899154268688692?s=46\n-\nhttps://x.com/pikamooncoin/status/1814174297356980706?s=46\n-\nTeam Meal \n\nhttps://x.com/orbitcosmosvc/status/1812879236098920786?s=46\n-\nHello From Pakistan 🇵🇰 \n\nhttps://x.com/pikamooncoin/status/1812743500011864486?s=46\n-\nHead over to our X for the latest Open World update 😁\n\n@ pikamooncoin 💎\n-\n'

In [3]:
from transformers import pipeline
import torch

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
result = summarizer(raw_posts, max_length=130, min_length=30, do_sample=False)
print(result[0]['summary_text'])

c:\Users\anush\OneDrive\Desktop\Work\smallcap\scraper-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Head over to our X for the latest Open World update. Here are the last few posts from a crypto token: Another Sneak Peak, Hello From Pakistan and Team Meal.
